# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [57]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [58]:
csv_path = "output.csv"
part1= pd.read_csv(csv_path)
clean=part1.drop(columns=["Unnamed: 0"])
clean.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ilulissat,69.2167,-51.1000,16.01,51,75,2.57,GL,1627409244
1,tocopilla,-22.0920,-70.1979,18.61,63,9,4.06,CL,1627409608
2,chokurdakh,70.6333,147.9167,1.64,86,100,4.96,RU,1627409608
3,bandarbeyla,9.4942,50.8122,26.08,73,98,12.18,SO,1627409347
4,ushuaia,-54.8000,-68.3000,2.81,87,90,2.06,AR,1627409577


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [59]:
 #configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = clean[["Lat", "Lng"]].astype(float)


#Add Heatmap layer to map
humidity = clean["Humidity"].astype(float)


In [60]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:
ideal_df=clean.loc[(clean["Wind Speed"] <= 10) & (clean["Cloudiness"]== 0) & \
                  (clean["Max Temp"] >= 25) & (clean["Max Temp"] <= 28)].dropna()
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
130,georgiyevka,43.0369,74.7114,27.02,28,0,6.00,KZ,1627409636
148,sulechow,52.0836,15.6251,26.53,50,0,3.09,PL,1627409640
162,lata,40.1629,-8.3327,26.69,53,0,5.13,PT,1627409644
181,rosetta,31.4044,30.4164,26.47,75,0,5.75,EG,1627409648
188,amposta,40.7131,0.5810,27.35,59,0,4.67,ES,1627409390
201,tongzhou,39.9053,116.5994,25.55,88,0,1.00,CN,1627409652
235,surt,31.2089,16.5887,27.03,66,0,7.33,LY,1627409662
263,sao joao da barra,-21.6403,-41.0511,27.05,67,0,6.29,BR,1627409482
295,benghazi,32.1167,20.0667,26.98,66,0,7.36,LY,1627409677
348,namsos,64.4662,11.4957,27.03,44,0,3.09,NO,1627409690


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
hotel_df = ideal_df.loc[:,["City","Country","Lat","Lng"]]


hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
130,georgiyevka,KZ,43.0369,74.7114,
148,sulechow,PL,52.0836,15.6251,
162,lata,PT,40.1629,-8.3327,
181,rosetta,EG,31.4044,30.4164,
188,amposta,ES,40.7131,0.5810,
201,tongzhou,CN,39.9053,116.5994,
235,surt,LY,31.2089,16.5887,
263,sao joao da barra,BR,-21.6403,-41.0511,
295,benghazi,LY,32.1167,20.0667,
348,namsos,NO,64.4662,11.4957,


In [63]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for City {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']

    #Save the hotel name to the dataframe
    try:
        print(f"The closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Could not find hotels within 5000 meters")
        print("-------------------------")
        
    print("--------------------Ending Search")

Retrieving Results for City 130: georgiyevka.
Could not find hotels within 5000 meters
-------------------------
--------------------Ending Search
Retrieving Results for City 148: sulechow.
The closest hotel in sulechow is Hotel Texicana.
--------------------Ending Search
Retrieving Results for City 162: lata.
The closest hotel in lata is Quinta do Outeiro.
--------------------Ending Search
Retrieving Results for City 181: rosetta.
The closest hotel in rosetta is فندق رشيد الدولي.
--------------------Ending Search
Retrieving Results for City 188: amposta.
The closest hotel in amposta is Lo Molí de Rosquilles.
--------------------Ending Search
Retrieving Results for City 201: tongzhou.
The closest hotel in tongzhou is Sunflower Hotel.
--------------------Ending Search
Retrieving Results for City 235: surt.
The closest hotel in surt is Al- Waffaa Hotel.
--------------------Ending Search
Retrieving Results for City 263: sao joao da barra.
The closest hotel in sao joao da barra is Pousada 

In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))